In [41]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
from geopy.distance import geodesic

india_map = gpd.read_file('C:/Users/lakshya.vashisth/Documents/Assignments/AI/Map_data/india_taluk.geojson')

# countries you want
states = ['Manipur', 'Nagaland', 'Assam']

filtered = []

for geo in india_map.values:
  if geo[4] in states:
    filtered.append(geo)

taluka_map = pd.DataFrame(filtered, columns =india_map.columns)
taluka_map = gpd.GeoDataFrame(taluka_map)
taluka_map["centroid"] = taluka_map.geometry.centroid

taluka_map.head()

,ID_0,ISO,NAME_0,ID_1,NAME_1,ID_2,NAME_2,ID_3,NAME_3,NL_NAME_3,VARNAME_3,TYPE_3,ENGTYPE_3,geometry,centroid
0,105,IND,India,4,Assam,41,Barpeta,228,Berpeta,None,Barpeta,Taluk,Taluk,"POLYGON ((91.01208 26.78564, 91.01382 26.78544...",POINT (91.02565 26.43871)
1,105,IND,India,4,Assam,42,Bongaigaon,229,Kokrajhar,None,None,Taluk,Taluk,"POLYGON ((90.95476 26.78384, 90.95339 26.77962...",POINT (90.64636 26.48985)
2,105,IND,India,4,Assam,43,Cachar,230,Silchar,None,None,Taluk,Taluk,"POLYGON ((92.4259 25.03182, 92.43192 25.03291,...",POINT (92.85752 24.81378)
3,105,IND,India,4,Assam,44,Darrang,231,Mangaldai,None,Darrang,Taluk,Taluk,"POLYGON ((92.36443 26.9327, 92.35907 26.92607,...",POINT (92.02641 26.60622)
4,105,IND,India,4,Assam,45,Dhemaji,232,Dhemaji,None,None,Taluk,Taluk,"POLYGON ((95.46165 27.79401, 95.46117 27.78775...",POINT (94.78022 27.59036)


In [42]:
G = nx.Graph()
pos = {}  # Stores positions (longitude, latitude)
labels = {}  # Stores Taluka names

for index, row in taluka_map.iterrows():
    centroid_x = row.centroid.x  # Longitude
    centroid_y = row.centroid.y  # Latitude
    # taluka_name = row.get("NAME_3", f"Taluka_{index}")  # Get taluka name from GeoJSON
    taluka_name = row.NAME_3
    G.add_node(taluka_name, pos=(centroid_x, centroid_y), name=taluka_name)
    pos[taluka_name] = (centroid_x, centroid_y)
    labels[taluka_name] = taluka_name  # Store label

edge_labels = {}
threshold_distance = 100  # in kilometers
for i, row1 in taluka_map.iterrows():
    for j, row2 in taluka_map.iterrows():
        if i != j and row1.geometry.touches(row2.geometry):
            coord1 = (row1.centroid.y, row1.centroid.x)
            coord2 = (row2.centroid.y, row2.centroid.x)
            distance = geodesic(coord1, coord2).km
            G.add_edge(row1.NAME_3, row2.NAME_3, weight=round(2*distance, 2))
            edge_labels[row1.NAME_3, row2.NAME_3] = f"{round(2*distance, 2)}"



In [43]:
from prettytable import PrettyTable
from geopy.distance import geodesic

heuristic_backward={}
heuristic_forward={}
# take start and end node
## alternative karimgang to Tuensang
start = "Berpeta"
goal = "Tuensang"

row_goal = taluka_map[taluka_map['NAME_3'] == goal].iloc[0]
row_start = taluka_map[taluka_map['NAME_3'] == start].iloc[0]

for i, row in taluka_map.iterrows():
  coord1 = (row.centroid.y, row.centroid.x)
  coord2 = (row_goal.centroid.y, row_goal.centroid.x)
  heuristic_backward[row.NAME_3] = round(geodesic(coord1, coord2).km, 2)

for i, row in taluka_map.iterrows():
  coord1 = (row.centroid.y, row.centroid.x)
  coord2 = (row_start.centroid.y, row_start.centroid.x)
  heuristic_forward[row.NAME_3] = round(geodesic(coord1, coord2).km, 2)

print("Heuristic Table with Start Node as",start, "and Goal Node as",goal)
t = PrettyTable(['Taluka', 'Cost'])
for key, value in heuristic_backward.items():
  t.add_row([key, value])
print(t)

print("\n Heuristic Table with Start Node as",goal, "and Goal Node as",start)
t1 = PrettyTable(['Taluka', 'Cost'])
for key, value in heuristic_forward.items():
  t1.add_row([key, value])
print(t1)

Heuristic Table with Start Node as Berpeta and Goal Node as Tuensang
+---------------+--------+
|     Taluka    |  Cost  |
+---------------+--------+
|    Berpeta    | 384.1  |
|   Kokrajhar   | 467.75 |
|    Silchar    | 248.86 |
|   Mangaldai   | 287.24 |
|    Dhemaji    | 161.17 |
|    Gauripur   | 482.88 |
|   Dibrugarh   | 138.13 |
|    Goalpara   | 425.11 |
|    Golaghat   | 108.67 |
|   Hailakandi  | 291.88 |
|     Jorhat    | 93.23  |
|    Guwahati   | 332.18 |
|     Diphu     | 145.02 |
|  n.a. ( 571)  | 234.73 |
|   Karimganj   | 303.21 |
|   Lakhimpur   | 135.53 |
|     Nagaon    | 201.71 |
|    Nalbari    | 344.38 |
|    Haflong    | 201.23 |
|    Sibsagar   |  97.0  |
|     Tezpur    | 205.29 |
|   Chapakhowa  | 210.6  |
|    Tinsukia   | 168.91 |
|   Bishnupur   | 198.32 |
|    Moirang    | 214.04 |
|  Chakpikarong | 245.88 |
|    Chandel    | 219.33 |
|   Tengnoupal  | 200.65 |
| Churachandpur | 231.43 |
|    Henglep    | 228.79 |
|    Parbung    | 263.65 |
|    Thanlon 

In [ ]:
import heapq

class BidirectionSearch:

    def __init__():
        self.fwd_
     
    def get_weight(G, node1, node2):
        return G.edges[node1, node2]['weight']

    def bidirectional_astar(graph, start, goal, heuristic_fwd, heuristic_bwd):

        frontier_cost = {}

        frontier_fwd = []
        frontier_bwd = []
        
        transition_cost_fwd = {start: 0}
        transition_cost_bwd = {goal: 0}
        
        # f(n) values (g(n) + h(n)) for forward and backward searches
        f_start = {start: transition_cost_fwd[start] + heuristic_fwd[start]}
        f_goal = {goal: transition_cost_bwd[goal] + heuristic_bwd[goal]}
        
        visited_start = {start: None}
        visited_goal = {goal: None}
        
        heapq.heappush(frontier_fwd, (f_start[start], start))
        heapq.heappush(frontier_bwd, (f_goal[goal], goal))
        current_start = start
        current_goal = goal
        
        while frontier_fwd and frontier_bwd:
            # Expand from the start node
                if frontier_fwd:
                    _, current_start = heapq.heappop(frontier_fwd)
                    print("Current forward node is: ",current_start)
                    for neighbor in graph[current_start].keys():
                        #print(f"neighbour: {neighbor}")
                        edge_cost = graph[current_start][neighbor]['weight']   # Fetch edge cost from the graph
                        #print(f"edge_cost for {current_start} to {neighbor}: {edge_cost}")
                        if neighbor not in visited_start:
                            transition_cost_fwd[neighbor] = transition_cost_fwd[current_start] + edge_cost #check this if we have to
                            f_start[neighbor] = transition_cost_fwd[neighbor] + heuristic_fwd[neighbor]
                            #print(f"Total cost: {f_start[neighbor]}")
                            visited_start[neighbor] = current_start
                            heapq.heappush(frontier_fwd, (f_start[neighbor], neighbor))
                            
                
                
                if frontier_bwd:
                    _, current_goal = heapq.heappop(frontier_bwd)
                    print("Current backward node is: ",current_goal)
                    for neighbor in graph[current_goal].keys():
                        #print(f"neighbour: {neighbor}")
                        edge_cost = graph[current_goal][neighbor]['weight']  # Fetch edge cost from the graph
                        #print(f"edge_cost for {current_goal} to {neighbor}: {edge_cost}")
                        if neighbor not in visited_goal:
                            transition_cost_bwd[neighbor] = transition_cost_bwd[current_goal] + edge_cost
                            f_goal[neighbor] = transition_cost_bwd[neighbor] + heuristic_bwd[neighbor]
                            #print(f"Total cost: {f_goal[neighbor]}")
                            visited_goal[neighbor] = current_goal
                            heapq.heappush(frontier_bwd, (f_goal[neighbor], neighbor))
            
        else:
            return reconstruct_path(visited_start, visited_goal, current_start)


    def reconstruct_path(visited_start, visited_goal, meet_node):
        # Reconstruct the path from start to the meet_node using the forward search
        path_start = []
        current = meet_node
        while current is not None:
            path_start.append(current)
            current = visited_start[current]
        
        # Reconstruct the path from the goal to the meet_node using the backward search
        path_goal = []
        current = visited_goal[meet_node]
        while current is not None:
            path_goal.append(current)
            current = visited_goal[current]
        
        return path_start[::-1] + path_goal


path = bidirectional_astar(G, start, goal, heuristic_backward, heuristic_forward)
print(f"Path: {path}")

Current forward node is:  Berpeta
Current backward node is:  Tuensang
Current forward node is:  Nalbari
Current backward node is:  Zunheboto
Current forward node is:  Guwahati
Current backward node is:  Mokokchung
Current forward node is:  Nagaon
Current backward node is:  Wokha
Current forward node is:  Mangaldai
Current backward node is:  Phek
Current forward node is:  Diphu
Current backward node is:  Jorhat
Current forward node is:  Kokrajhar
Current backward node is:  Mon
Current forward node is:  Goalpara
Current backward node is:  Golaghat
Current forward node is:  Golaghat
Current backward node is:  Henima
Current forward node is:  Tezpur
Current backward node is:  Chingai
Current forward node is:  n.a. ( 571)
Current backward node is:  Karong
Current forward node is:  Wokha
Current backward node is:  Sibsagar
Current forward node is:  Henima
Current backward node is:  Lakhimpur
Current forward node is:  Haflong
Current backward node is:  Diphu
Current forward node is:  Jorhat
C

KeyboardInterrupt: 

In [ ]:
import heapq

def bidirectional_astar(graph, start, goal, heuristic_fwd, heuristic_bwd):
    # Priority queues for forward and backward searches
    frontier_fwd = []
    frontier_bwd = []
    
    # Transition costs (g(n)) for forward and backward searches
    transition_cost_fwd = {start: 0}
    transition_cost_bwd = {goal: 0}
    
    # f(n) values (g(n) + h(n)) for forward and backward searches
    f_start = {start: heuristic_fwd[start]}
    f_goal = {goal: heuristic_bwd[goal]}
    
    # Visited nodes for forward and backward searches
    visited_start = []
    visited_goal = []

    fw_cost = 0
    bw_cost = 0
    
    # Push the start and goal nodes into their respective priority queues
    heapq.heappush(frontier_fwd, (f_start[start], start))
    heapq.heappush(frontier_bwd, (f_goal[goal], goal))
    
    meet_node = None  # Node where the two searches meet
    
    while frontier_fwd and frontier_bwd:
        # Forward search step
        if frontier_fwd:
            cost, current_start = heapq.heappop(frontier_fwd)
            if visited_goal and current_start == visited_goal[-1]:  # Check if the node is visited by backward search
                meet_node = current_start
                break
            visited_start.append(current_start)
            for neighbor in graph[current_start]:
                edge_cost = graph[current_start][neighbor]['weight']
                new_cost = transition_cost_fwd[current_start] + edge_cost
                
                if neighbor not in transition_cost_fwd or new_cost < transition_cost_fwd[neighbor]:
                    transition_cost_fwd[neighbor] = new_cost
                    f_start[neighbor] = new_cost + heuristic_fwd[neighbor]
                    heapq.heappush(frontier_fwd, (f_start[neighbor], neighbor))
        
        # Backward search step
        if frontier_bwd:
            cost, current_goal = heapq.heappop(frontier_bwd)
            if visited_start and current_start == visited_start[-1]:
                meet_node = current_goal
                break
            visited_goal.append(current_goal)
            for neighbor in graph[current_goal]:
                edge_cost = graph[current_goal][neighbor]['weight']
                new_cost = transition_cost_bwd[current_goal] + edge_cost
                
                if neighbor not in transition_cost_bwd or new_cost < transition_cost_bwd[neighbor]:
                    transition_cost_bwd[neighbor] = new_cost
                    f_goal[neighbor] = new_cost + heuristic_bwd[neighbor]
                    heapq.heappush(frontier_bwd, (f_goal[neighbor], neighbor))
    
    if meet_node is None:
        return None  
    
    return meet_node, reconstruct_path(visited_start, visited_goal, meet_node)



def reconstruct_path(visited_start, visited_goal, meet_node):
    # Reconstruct the path from start to meet_node using forward search
    path_start = []
    current = meet_node
    while current is not None :
        path_start.append(current)
        current = visited_start
    
    # Reconstruct the path from goal to meet_node using backward search
    path_goal = []
    current = meet_node
    while current is not None:
        path_goal.append(current)
        current = visited_goal.get(current)
    
    return path_start[::-1] + path_goal[1:]  



path = bidirectional_astar(G, start, goal, heuristic_forward, heuristic_backward)
print(f"Path: {path}")

KeyboardInterrupt: 

In [60]:
import heapq

class BidirectionalAStar:
    def __init__(self, graph, start, goal, heuristic_fwd, heuristic_bwd):
        self.graph = graph
        self.start = start
        self.goal = goal
        self.heuristic_fwd = heuristic_fwd
        self.heuristic_bwd = heuristic_bwd
        
        # Priority queues for forward and backward searches
        self.frontier_fwd = []
        self.frontier_bwd = []
        
        # Transition costs (g(n)) for forward and backward searches
        self.transition_cost_fwd = {start: 0}
        self.transition_cost_bwd = {goal: 0}
        
        # f(n) values (g(n) + h(n)) for forward and backward searches
        self.f_start = {start: heuristic_fwd[start]}
        self.f_goal = {goal: heuristic_bwd[goal]}
        
        # Visited nodes for forward and backward searches
        self.visited_start = []
        self.visited_goal = []
        
        # Total costs
        self.fw_cost = 0
        self.bw_cost = 0
        
        # Push the start and goal nodes into their respective priority queues
        heapq.heappush(self.frontier_fwd, (self.f_start[start], start))
        heapq.heappush(self.frontier_bwd, (self.f_goal[goal], goal))
        
        self.meet_node = None  # Node where the two searches meet
    
    def run(self):
        while self.frontier_fwd and self.frontier_bwd:
            # Forward search step
            if self.frontier_fwd:
                cost, current_start = heapq.heappop(self.frontier_fwd)
                if current_start in self.visited_goal:  # Check if the node is visited by backward search
                    self.meet_node = current_start
                    break
                self.visited_start.append(current_start)
                self.expand_node(self.frontier_fwd, self.transition_cost_fwd, self.f_start, 
                                 self.visited_start, current_start, self.heuristic_fwd, is_forward=True)
            
            # Backward search step
            if self.frontier_bwd:
                _, current_goal = heapq.heappop(self.frontier_bwd)
                if current_goal in self.visited_start:  # Check if the node is visited by forward search
                    self.meet_node = current_goal
                    break
                self.visited_goal.append(current_goal)
                self.expand_node(self.frontier_bwd, self.transition_cost_bwd, self.f_goal,
                                 self.visited_goal, current_goal, self.heuristic_bwd, is_forward=False)
        
        if self.meet_node is None:
            return None, 0  # No path found
        
        # Reconstruct the path and calculate its cost
        path = self.reconstruct_path(self.meet_node)
        total_cost = self.transition_cost_fwd[self.meet_node] + self.transition_cost_bwd[self.meet_node]
        
        return path, total_cost

    def expand_node(self, frontier, transition_cost, f_values, visited, current_node, heuristic, is_forward):
        # Expand a node in the search process
        for neighbor in self.graph[current_node]:
            edge_cost = self.graph[current_node][neighbor]['weight']
            new_cost = transition_cost[current_node] + edge_cost
            
            if neighbor not in transition_cost or new_cost < transition_cost[neighbor]:
                transition_cost[neighbor] = new_cost
                f_values[neighbor] = new_cost + heuristic[neighbor]
                heapq.heappush(frontier, (f_values[neighbor], neighbor))

    def reconstruct_path(self, meet_node):
        # Reconstruct the path from start to meet_node using forward search
        path_start = []
        current = meet_node
        while current != self.start:
            path_start.append(current)
            current = self.visited_start[self.visited_start.index(current)-1] if current in self.visited_start else None

        # Reconstruct the path from goal to meet_node using backward search
        path_goal = []
        current = meet_node
        while current != self.goal:
            path_goal.append(current)
            current = self.visited_goal[self.visited_goal.index(current)-1] if current in self.visited_goal else None
        
        return path_start[::-1] + path_goal[1:]

# Example usage
graph = {
    'A': {'B': {'weight': 1}, 'C': {'weight': 3}},
    'B': {'A': {'weight': 1}, 'C': {'weight': 1}, 'D': {'weight': 3}},
    'C': {'A': {'weight': 3}, 'B': {'weight': 1}, 'D': {'weight': 1}},
    'D': {'B': {'weight': 3}, 'C': {'weight': 1}}
}
heuristic_fwd = {'A': 2, 'B': 1, 'C': 0, 'D': 3}
heuristic_bwd = {'A': 3, 'B': 2, 'C': 1, 'D': 0}

# Create an instance of BidirectionalAStar
bida_star = BidirectionalAStar(graph, 'A', 'D', heuristic_fwd, heuristic_bwd)

# Run the bidirectional A* algorithm
path, cost = bida_star.run()

print(f"Path: {path}")
print(f"Total cost of travel: {cost}")


KeyboardInterrupt: 

In [ ]:
import heapq

def bidirectional_astar(graph, start, goal, heuristic_fwd, heuristic_bwd):
    # Priority queues for forward and backward searches
    frontier_fwd = []
    frontier_bwd = []
    
    # Transition costs (g(n)) for forward and backward searches
    transition_cost_fwd = {start: 0}
    transition_cost_bwd = {goal: 0}
    
    # f(n) values (g(n) + h(n)) for forward and backward searches
    f_start = {start: heuristic_fwd[start]}
    f_goal = {goal: heuristic_bwd[goal]}
    
    # Visited nodes for forward and backward searches (stores parent nodes)
    visited_start = {start: None}
    visited_goal = {goal: None}
    
    # Push the start and goal nodes into their respective priority queues
    heapq.heappush(frontier_fwd, (f_start[start], start))
    heapq.heappush(frontier_bwd, (f_goal[goal], goal))
    
    meet_node = None  # Node where the two searches meet
    
    while frontier_fwd and frontier_bwd:
        # Forward search step
        if len(frontier_fwd) <= len(frontier_bwd):
            _, current_start = heapq.heappop(frontier_fwd)
            
            if current_start in visited_goal:  # Check if the node is visited by backward search
                meet_node = current_start
                break
            
            for neighbor in graph[current_start]:
                edge_cost = graph[current_start][neighbor]['weight']
                new_cost = transition_cost_fwd[current_start] + edge_cost
                
                if neighbor not in transition_cost_fwd or new_cost < transition_cost_fwd[neighbor]:
                    transition_cost_fwd[neighbor] = new_cost
                    f_start[neighbor] = new_cost + heuristic_fwd[neighbor]
                    visited_start[neighbor] = current_start  # Record parent node
                    heapq.heappush(frontier_fwd, (f_start[neighbor], neighbor))
        
        # Backward search step
        else:
            _, current_goal = heapq.heappop(frontier_bwd)
            
            if current_goal in visited_start:  # Check if the node is visited by forward search
                meet_node = current_goal
                break
            
            for neighbor in graph[current_goal]:
                edge_cost = graph[current_goal][neighbor]['weight']
                new_cost = transition_cost_bwd[current_goal] + edge_cost
                
                if neighbor not in transition_cost_bwd or new_cost < transition_cost_bwd[neighbor]:
                    transition_cost_bwd[neighbor] = new_cost
                    f_goal[neighbor] = new_cost + heuristic_bwd[neighbor]
                    visited_goal[neighbor] = current_goal  
                    heapq.heappush(frontier_bwd, (f_goal[neighbor], neighbor))
    
    # If the meeting node is found, we reconstruct the path
    if meet_node is None:
        return None, [], []
    
    # Reconstruct the full path (start to meet node, then meet node to goal)
    path_start = []
    current = meet_node
    while current is not None:
        path_start.append(current)
        current = visited_start.get(current)  # Get the parent node
    
    path_goal = []
    current = meet_node
    while current is not None:
        path_goal.append(current)
        current = visited_goal.get(current)  # Get the parent node
    
    full_path = path_start[::-1] + path_goal[1:]  # Full path from start to goal
    
    return meet_node, full_path, path_start[::-1], path_goal[::-1]


# Run the bidirectional A* algorithm
meet_node, full_path, path_fwd, path_bwd = bidirectional_astar(G, start, goal, heuristic_forward, heuristic_backward)

print(f"Meet Node: {meet_node}")
print(f"Full Path: {full_path}")
print(f"Forward Path (from start to meet): {path_fwd}")
print(f"Backward Path (from goal to meet): {path_bwd}")


Meet Node: Henima
Full Path: ['Berpeta', 'Guwahati', 'Nagaon', 'Diphu', 'Henima', 'Zunheboto', 'Tuensang']
Forward Path (from start to meet): ['Berpeta', 'Nalbari', 'Kokrajhar', 'Goalpara', 'Guwahati', 'Mangaldai', 'Nagaon', 'n.a. ( 571)', 'Tezpur', 'Diphu', 'Haflong', 'Golaghat', 'Henima']
Backward Path (from goal to meet): ['Tuensang', 'Zunheboto']
